In [2]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8,
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [4]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [5]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)



(array([0, 1]), array([619, 619]))

### Track Experiments

In [6]:
models = [
    (
        "Logistic Regression",
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest",
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(),
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [7]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]

    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [8]:
!pip install mlflow
import mlflow
import mlflow.sklearn
import mlflow.xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.9/705.9 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 10.6 MB/s eta 0:00:00


In [9]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection")
# mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]

    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })

        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")

        # if "XGB" in model_name:
        #     model_uri = f"runs:/{run_id}/model"
        #     mlflow.register_model(model_uri=model_uri, name=model_name)

        #     print(f"Model {model_name} registered with run_id: {run_id}")

2025/09/11 12:23:40 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection' does not exist. Creating a new experiment.
2025/09/11 12:23:40 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/11 12:23:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/09/11 12:23:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/11 12:23:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/09/11 12:23:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/11 12:24:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the

### Register the Model

In [10]:
# model_uri = f"runs:/{run_id}/model"
# mlflow.register_model(model_uri=model_uri, name=model_name)
# print(f"Model {model_name} registered with run_id: {run_id}")

In [11]:
import mlflow

mlflow.set_experiment("Anomaly Detection")
# Optional: set your tracking URI if remote
# mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]

    with mlflow.start_run(run_name=model_name) as run:
        run_id = run.info.run_id

        # Log parameters and metrics
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })

        # Log model
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")

        # ✅ Register the model (inside the active run)
        model_uri = f"runs:/{run_id}/model"
        mlflow.register_model(model_uri=model_uri, name=model_name)

        print(f"✅ Model {model_name} registered successfully with run_id: {run_id}")


2025/09/11 12:24:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/11 12:24:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Logistic Regression'.
2025/09/11 12:24:13 WARNING mlflow.tracking._model_registry.fluent: Run with id 8d4f0b11b9fe4501ade58085b09edaf5 has no artifacts at artifact path 'model', registering model based on models:/m-8bec74fbddde4dec923f95ed66d54426 instead
Created version '1' of model 'Logistic Regression'.
2025/09/11 12:24:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✅ Model Logistic Regression registered successfully with run_id: 8d4f0b11b9fe4501ade58085b09edaf5


2025/09/11 12:24:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Random Forest'.
2025/09/11 12:24:21 WARNING mlflow.tracking._model_registry.fluent: Run with id 8323962e26da4cc28e915681393c88f4 has no artifacts at artifact path 'model', registering model based on models:/m-20ef85ee28f142f99c30089fee61c1ee instead
Created version '1' of model 'Random Forest'.
2025/09/11 12:24:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✅ Model Random Forest registered successfully with run_id: 8323962e26da4cc28e915681393c88f4


2025/09/11 12:24:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'XGBClassifier'.
2025/09/11 12:24:29 WARNING mlflow.tracking._model_registry.fluent: Run with id 84db9e11f6a648cca868159657701708 has no artifacts at artifact path 'model', registering model based on models:/m-51cda47480824912bfc3325f608338a7 instead
Created version '1' of model 'XGBClassifier'.
2025/09/11 12:24:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✅ Model XGBClassifier registered successfully with run_id: 84db9e11f6a648cca868159657701708


2025/09/11 12:24:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'XGBClassifier With SMOTE'.
2025/09/11 12:24:35 WARNING mlflow.tracking._model_registry.fluent: Run with id 3208104cc28048faa12d7e6d3e82ce90 has no artifacts at artifact path 'model', registering model based on models:/m-5d3ce4fb8f994ba0a60e7e418a5f49aa instead


✅ Model XGBClassifier With SMOTE registered successfully with run_id: 3208104cc28048faa12d7e6d3e82ce90


Created version '1' of model 'XGBClassifier With SMOTE'.


In [12]:
mlflow.register_model(model_uri=f"runs:/{run_id}/model", name="XGB-Smote")


Successfully registered model 'XGB-Smote'.
2025/09/11 12:24:35 WARNING mlflow.tracking._model_registry.fluent: Run with id 3208104cc28048faa12d7e6d3e82ce90 has no artifacts at artifact path 'model', registering model based on models:/m-5d3ce4fb8f994ba0a60e7e418a5f49aa instead
Created version '1' of model 'XGB-Smote'.


<ModelVersion: aliases=[], creation_timestamp=1757593475243, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1757593475243, metrics=[<Metric: dataset_digest=None, dataset_name=None, key='accuracy', model_id='m-5d3ce4fb8f994ba0a60e7e418a5f49aa', run_id='3208104cc28048faa12d7e6d3e82ce90', step=0, timestamp=1757593470065, value=0.9633333333333334>,
 <Metric: dataset_digest=None, dataset_name=None, key='f1_score_macro', model_id='m-5d3ce4fb8f994ba0a60e7e418a5f49aa', run_id='3208104cc28048faa12d7e6d3e82ce90', step=0, timestamp=1757593470065, value=0.8996319839411174>,
 <Metric: dataset_digest=None, dataset_name=None, key='recall_class_0', model_id='m-5d3ce4fb8f994ba0a60e7e418a5f49aa', run_id='3208104cc28048faa12d7e6d3e82ce90', step=0, timestamp=1757593470065, value=0.9777777777777777>,
 <Metric: dataset_digest=None, dataset_name=None, key='recall_class_1', model_id='m-5d3ce4fb8f994ba0a60e7e418a5f49aa', run_id='3208104cc28048faa12d7e6d3e82ce90', step

### Load the Model

In [13]:
import mlflow.xgboost

model_name = "XGB-Smote"
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)

# Predict
y_pred = loaded_model.predict(X_test)
print(y_pred[:])


[0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1
 1 0 0 0]


In [14]:
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

### Transition the Model to Production

In [15]:
from mlflow.tracking import MlflowClient

model_name = "XGB-Smote"
production_model_name = "anomaly-detection-prod"
source_version = 1

client = MlflowClient()

# (Optional) set alias for clarity
client.set_registered_model_alias(model_name, "challenger", version=source_version)

# Make sure destination exists
try:
    client.create_registered_model(production_model_name)
except Exception:
    pass  # Already exists

# Copy the version
client.copy_model_version(
    src_model_uri=f"models:/{model_name}@challenger",
    dst_name=production_model_name
)


Registered model 'anomaly-detection-prod' already exists. Creating a new version of this model...
Copied version '1' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1757593475335, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1757593475335, metrics=None, model_id=None, name='anomaly-detection-prod', params=None, run_id='3208104cc28048faa12d7e6d3e82ce90', run_link=None, source='models:/XGB-Smote/1', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [16]:
import mlflow
from mlflow.tracking import MlflowClient


model_name = "XGB-Smote"                # Source model
production_model_name = "anomaly-detection-prod"  # Destination registry
source_version = 1

client = MlflowClient()

# Set alias "challenger" on source model
client.set_registered_model_alias(model_name, "challenger", version=source_version)

# Make sure destination model registry exists
try:
    client.create_registered_model(production_model_name)
except Exception:
    pass  # Already exists

# Copy challenger into production registry
dst_model_version = client.copy_model_version(
    src_model_uri=f"models:/{model_name}@challenger",
    dst_name=production_model_name
)

# Set alias "champion" on the copied version
client.set_registered_model_alias(
    production_model_name,
    "champion",
    version=dst_model_version.version
)

#Load champion model & predict
prod_model_uri = f"models:/{production_model_name}@champion"
loaded_model = mlflow.xgboost.load_model(prod_model_uri)

y_pred = loaded_model.predict(X_test)
print(y_pred[:4])

Registered model 'anomaly-detection-prod' already exists. Creating a new version of this model...
Copied version '1' of model 'XGB-Smote' to version '2' of model 'anomaly-detection-prod'.


[0 0 0 0]


In [18]:
import mlflow

# Download the model from MLflow Model Registry
export_path = mlflow.artifacts.download_artifacts(
    artifact_uri="models:/anomaly-detection-prod@champion",
    dst_path="exported_model"
)

print("Model exported to:", export_path)



Model exported to: /content/exported_model/


Create a Serving Script (FastAPI or Flask)

In [19]:
from fastapi import FastAPI
import mlflow.pyfunc

app = FastAPI()

# Load exported model
model = mlflow.pyfunc.load_model("exported_model")

@app.get("/")
def root():
    return {"message": "MLflow model is live inside Docker!"}

@app.post("/predict")
def predict(data: dict):
    X = [list(data.values())]
    y_pred = model.predict(X)
    return {"prediction": y_pred.tolist()}


In [20]:
%%writefile serve.py
from fastapi import FastAPI
import mlflow
import pandas as pd

# Create FastAPI app
app = FastAPI()

# Load model from MLflow registry (champion alias)
model_uri = "models:/anomaly-detection-prod@champion"
model = mlflow.pyfunc.load_model(model_uri)

@app.get("/")
def home():
    return {"message": "MLflow model is live inside Docker!"}

@app.post("/predict")
def predict(data: dict):
    """
    Expects input like:
    {
        "feature1": 0.5,
        "feature2": 1.2,
        "feature3": -0.8
    }
    """
    # Convert input dict → DataFrame
    df = pd.DataFrame([data])

    # Predict
    prediction = model.predict(df)

    return {"prediction": prediction.tolist()}


Writing serve.py


In [21]:
from google.colab import files
files.download("serve.py")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Creating Docker file

In [22]:
%%writefile Dockerfile
# Base image with Python
FROM python:3.9

# Set working directory inside container
WORKDIR /app

# Copy files
COPY serve.py /app
COPY requirements.txt /app

# Install dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Expose FastAPI port
EXPOSE 8000

# Start FastAPI app
CMD ["uvicorn", "serve:app", "--host", "0.0.0.0", "--port", "8000"]


Writing Dockerfile


In [23]:
from google.colab import files
files.download("Dockerfile")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
mlflow
fastapi
uvicorn
scikit-learn
xgboost
